In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [4]:
data = pd.read_csv("/content/drive/MyDrive/Consulta_hospitales.csv", encoding ='ISO-8859-1')

In [5]:
data.head()

,ano,mes,sexo,ppertenencia,edad,periodoeda,deptoresiden,muniresiden,tc,caufin,source_file
0,2023,2,1,4,5,3,1,101,1,A010,bdexterna2023.xlsx
1,2023,8,1,9,9,3,1,101,1,A010,bdexterna2023.xlsx
2,2023,8,1,4,13,3,1,101,1,A010,bdexterna2023.xlsx
3,2023,7,1,9,14,3,1,101,2,A010,bdexterna2023.xlsx
4,2023,9,1,9,16,3,1,101,3,A010,bdexterna2023.xlsx


In [14]:
data = data.drop("source_file", axis=1, errors='ignore')
data = data.rename(columns={
    "ppertenencia": "pueblo_pertenencia",
    "periodoeda": "periodo_edad",
    "muniresiden": "municipio_residencia",
    "tc": "tipo_consulta",
    "caufin": "causa_atencion"
})

In [9]:
data = data.rename(columns={
    "ppertenencia": "pueblo_pertenencia",
    "periodoeda": "periodo_edad",
    "muniresiden": "municipio_residencia",
    "tc": "tipo_consulta",
    "caufin": "causa_atencion"
})

In [10]:
data.head()

,ano,mes,sexo,pueblo_pertenencia,edad,periodo_edad,deptoresiden,municipio_residencia,tipo_consulta,causa_atencion
0,2023,2,1,4,5,3,1,101,1,A010
1,2023,8,1,9,9,3,1,101,1,A010
2,2023,8,1,4,13,3,1,101,1,A010
3,2023,7,1,9,14,3,1,101,2,A010
4,2023,9,1,9,16,3,1,101,3,A010


# **Caso 1: Consultas de Emergencia en la Capital**


In [12]:
datos_mayores_emergencias = data[
    (data["tipo_consulta"] == 3) &
    (data["deptoresiden"] == 1) &
    (data["causa_atencion"] != "A099")
]

In [15]:
x = datos_mayores_emergencias[['edad','mes','sexo','deptoresiden']]
x = x.fillna(0)
y, _ = pd.factorize(datos_mayores_emergencias['causa_atencion'])

In [16]:
x_train, x_test, y_train ,y_test = train_test_split(x,y, test_size=0.2, random_state=42)

In [25]:
num_classes = len(np.unique(y))
model = Sequential()
model.add(Dense(16, input_dim=3, activation='relu'))
model.add(Dense(16,activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [26]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [27]:
model.fit(x_train, y_train, epochs=50, batch_size=64, validation_data=(x_test, y_test))

Epoch 1/50
1809/1809 ━━━━━━━━━━━━━━━━━━━━ 18s 9ms/step - accuracy: 0.0890 - loss: 5.5447 - val_accuracy: 0.1126 - val_loss: 4.9373
Epoch 2/50
1809/1809 ━━━━━━━━━━━━━━━━━━━━ 16s 9ms/step - accuracy: 0.1124 - loss: 4.9237 - val_accuracy: 0.1135 - val_loss: 4.9015
Epoch 3/50
1809/1809 ━━━━━━━━━━━━━━━━━━━━ 15s 8ms/step - accuracy: 0.1130 - loss: 4.8788 - val_accuracy: 0.1152 - val_loss: 4.8956
Epoch 4/50
1809/1809 ━━━━━━━━━━━━━━━━━━━━ 16s 9ms/step - accuracy: 0.1144 - loss: 4.8644 - val_accuracy: 0.1169 - val_loss: 4.8877
Epoch 5/50
1809/1809 ━━━━━━━━━━━━━━━━━━━━ 15s 9ms/step - accuracy: 0.1166 - loss: 4.8337 - val_accuracy: 0.1179 - val_loss: 4.8844
Epoch 6/50
1809/1809 ━━━━━━━━━━━━━━━━━━━━ 16s 9ms/step - accuracy: 0.1172 - loss: 4.8160 - val_accuracy: 0.1178 - val_loss: 4.8917
Epoch 7/50
1809/1809 ━━━━━━━━━━━━━━━━━━━━ 17s 9ms/step - accuracy: 0.1157 - loss: 4.8189 - val_accuracy: 0.1168 - val_loss: 4.8874
Epoch 8/50
1809/1809 ━━━━━━━━━━━━━━━━━━━━ 16s 9ms/step - accuracy: 0.1173 - loss: 4

In [28]:
loss, accuracy = model.evaluate(x_test, y_test)
print(f'Loss: {loss}, Accuracy: {accuracy}')

905/905 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.1192 - loss: 5.0236
Loss: 5.028038501739502, Accuracy: 0.12000691145658493


In [86]:
#Perfil 1: Mujer de 25 años, enferma en octubre
Perfil1 = np.array([[25,10,2]])
res = model.predict(Perfil1)
print(res)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
[[0.03924764]]


In [87]:
#Perfil 2: Hombre de 10 años, enfermo en julio
Perfil2 = np.array([[10,7,1]])
res = model.predict(Perfil1)
print(res)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
[[0.03924764]]


In [88]:
#Perfil 3: Mujer de 5 años, enferma en enero
Perfil3 = np.array([[5,1,2]])
res = model.predict(Perfil1)
print(res)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
[[0.03924764]]


# Caso 2: Diagnóstico de Acné Vulgar y Asociación con Pueblo de Pertenencia

Sin balancear los datos, se obtuvo un desbalanceo, ya que la mayoría de datos eran No_L700

In [33]:
datos_xinka = data[['causa_atencion', 'sexo', 'pueblo_pertenencia', 'tipo_consulta']].copy()

In [34]:
datos_xinka['causa'] = np.where(datos_xinka['causa_atencion'] == "L700", 1, 0)
datos_xinka = datos_xinka.dropna()

In [35]:
datos_xinka =datos_xinka.sample(frac=1, random_state=42).reset_index(drop='TRUE')

In [37]:
x = datos_xinka[['pueblo_pertenencia','sexo','tipo_consulta']]
x = x.fillna(0)
y = datos_xinka['causa']

In [38]:
x_train, x_test, y_train ,y_test = train_test_split(x,y, test_size=0.2, random_state=42)

In [43]:
model = Sequential()
model.add(Dense(8, input_dim=3, activation='relu'))
model.add(Dense(4,activation='relu'))
model.add(Dense(1, activation='sigmoid'))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [44]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [45]:
model.fit(x_train, y_train, epochs=50, batch_size=64, validation_data=(x_test, y_test))

Epoch 1/50
83273/83273 ━━━━━━━━━━━━━━━━━━━━ 244s 3ms/step - accuracy: 0.9954 - loss: 0.0294 - val_accuracy: 0.9968 - val_loss: 0.0208
Epoch 2/50
83273/83273 ━━━━━━━━━━━━━━━━━━━━ 259s 3ms/step - accuracy: 0.9969 - loss: 0.0205 - val_accuracy: 0.9968 - val_loss: 0.0209
Epoch 3/50
83273/83273 ━━━━━━━━━━━━━━━━━━━━ 232s 3ms/step - accuracy: 0.9968 - loss: 0.0207 - val_accuracy: 0.9968 - val_loss: 0.0209
Epoch 4/50
83273/83273 ━━━━━━━━━━━━━━━━━━━━ 244s 3ms/step - accuracy: 0.9968 - loss: 0.0206 - val_accuracy: 0.9968 - val_loss: 0.0207
Epoch 5/50
83273/83273 ━━━━━━━━━━━━━━━━━━━━ 262s 3ms/step - accuracy: 0.9968 - loss: 0.0207 - val_accuracy: 0.9968 - val_loss: 0.0207
Epoch 6/50
83273/83273 ━━━━━━━━━━━━━━━━━━━━ 241s 3ms/step - accuracy: 0.9968 - loss: 0.0207 - val_accuracy: 0.9968 - val_loss: 0.0207
Epoch 7/50
83273/83273 ━━━━━━━━━━━━━━━━━━━━ 282s 3ms/step - accuracy: 0.9968 - loss: 0.0207 - val_accuracy: 0.9968 - val_loss: 0.0207
Epoch 8/50
83273/83273 ━━━━━━━━━━━━━━━━━━━━ 244s 3ms/step - ac

In [46]:
loss, accuracy = model.evaluate(x_test, y_test)
print(f'Loss: {loss}, Accuracy: {accuracy}')

41637/41637 ━━━━━━━━━━━━━━━━━━━━ 95s 2ms/step - accuracy: 0.9968 - loss: 0.0204
Loss: 0.020539699122309685, Accuracy: 0.9968161582946777


Se balancearon los datos debido que en general, los casos de Acné vulgar eran casi 0 en todo el dataset

In [49]:
idx_1 = datos_xinka[datos_xinka['causa'] == 1].index   # Si_L700
idx_0 = datos_xinka[datos_xinka['causa'] == 0].index   # No_L700

n_1 = len(idx_1)
n_0_mostrar = min(len(idx_0), n_1 * 4)

np.random.seed(123)
idx_0_sub = np.random.choice(idx_0, n_0_mostrar, replace=False)

In [50]:
datos_bal = datos_xinka.loc[np.concatenate([idx_1, idx_0_sub])]

# mezclar filas
datos_bal = datos_bal.sample(frac=1, random_state=42).reset_index(drop=True)

print(datos_bal['causa'].value_counts())

causa
0    84372
1    21093
Name: count, dtype: int64


In [51]:
X = datos_bal[['pueblo_pertenencia', 'sexo', 'tipo_consulta']]
y = datos_bal['causa']  # 0 / 1

X = X.fillna(0)

x_train, x_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [52]:
model = Sequential()
model.add(Dense(8, input_dim=3, activation='relu'))
model.add(Dense(4, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [53]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [54]:
model.fit(x_train, y_train, epochs=50, batch_size=64, validation_data=(x_test, y_test))

Epoch 1/50
1319/1319 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.6877 - loss: 0.6187 - val_accuracy: 0.7983 - val_loss: 0.5046
Epoch 2/50
1319/1319 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.8004 - loss: 0.5010 - val_accuracy: 0.7983 - val_loss: 0.4946
Epoch 3/50
1319/1319 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7987 - loss: 0.4937 - val_accuracy: 0.7978 - val_loss: 0.4889
Epoch 4/50
1319/1319 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7996 - loss: 0.4859 - val_accuracy: 0.7983 - val_loss: 0.4842
Epoch 5/50
1319/1319 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.8008 - loss: 0.4778 - val_accuracy: 0.7983 - val_loss: 0.4776
Epoch 6/50
1319/1319 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8011 - loss: 0.4720 - val_accuracy: 0.7983 - val_loss: 0.4750
Epoch 7/50
1319/1319 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7996 - loss: 0.4713 - val_accuracy: 0.7983 - val_loss: 0.4726
Epoch 8/50
1319/1319 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.7985 - loss: 0.4714 - 

In [57]:
loss, acc = model.evaluate(x_test, y_test)
print("Loss:", loss, "Accuracy:", acc)

660/660 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7981 - loss: 0.4664
Loss: 0.4645001292228699 Accuracy: 0.7983217239379883


In [89]:
#Perfil 1: Mujer del pueblo Xinka, en reconsulta
Perfil1 = np.array([[3,2,2]])
res = model.predict(Perfil1)
print(res)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
[[0.07979964]]


In [90]:
#Perfil 2: Hombre del pueblo Xinka, en reconsulta
Perfil2 = np.array([[3,1,2]])
res = model.predict(Perfil2)
print(res)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
[[0.21957833]]


In [91]:
#Perfil 3: Mujer maya, en reconsulta
Perfil3 = np.array([[1,2,2]])
res = model.predict(Perfil3)
print(res)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
[[0.3329031]]


# **Caso 3: Factores Sociodemográficos Asociados a Consultas de Emergencia**

In [72]:
datos_emergencia = data[['deptoresiden', 'sexo', 'pueblo_pertenencia', 'tipo_consulta']].copy()

In [73]:
datos_emergencia['consulta'] = np.where(datos_emergencia['tipo_consulta'] == 3, 1, 0)
datos_emergencia = datos_emergencia.dropna()

In [74]:
idx_1 = datos_emergencia[datos_emergencia['consulta'] == 1].index
idx_0 = datos_emergencia[datos_emergencia['consulta'] == 0].index

n_1 = len(idx_1)
n_0_mostrar = min(len(idx_0), n_1 * 4)

np.random.seed(123)
idx_0_sub = np.random.choice(idx_0, n_0_mostrar, replace=False)

In [75]:
datos_bal2 = datos_emergencia.loc[np.concatenate([idx_1, idx_0_sub])]

# mezclar filas
datos_bal2 = datos_bal2.sample(frac=1, random_state=42).reset_index(drop=True)

print(datos_bal2['consulta'].value_counts())

consulta
0    1622788
1     405697
Name: count, dtype: int64


In [77]:
X = datos_bal2[['deptoresiden', 'pueblo_pertenencia', 'sexo']]
y = datos_bal2['consulta']  # 0 / 1

X = X.fillna(0)

x_train, x_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [78]:
model = Sequential()
model.add(Dense(8, input_dim=3, activation='relu'))
model.add(Dense(4, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [79]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [80]:
model.fit(x_train, y_train, epochs=50, batch_size=64, validation_data=(x_test, y_test))

Epoch 1/50
25357/25357 ━━━━━━━━━━━━━━━━━━━━ 71s 3ms/step - accuracy: 0.8001 - loss: 0.4904 - val_accuracy: 0.7998 - val_loss: 0.4825
Epoch 2/50
25357/25357 ━━━━━━━━━━━━━━━━━━━━ 67s 3ms/step - accuracy: 0.8010 - loss: 0.4807 - val_accuracy: 0.7990 - val_loss: 0.4824
Epoch 3/50
25357/25357 ━━━━━━━━━━━━━━━━━━━━ 68s 3ms/step - accuracy: 0.8001 - loss: 0.4815 - val_accuracy: 0.7998 - val_loss: 0.4820
Epoch 4/50
25357/25357 ━━━━━━━━━━━━━━━━━━━━ 66s 3ms/step - accuracy: 0.8004 - loss: 0.4809 - val_accuracy: 0.7998 - val_loss: 0.4819
Epoch 5/50
25357/25357 ━━━━━━━━━━━━━━━━━━━━ 66s 3ms/step - accuracy: 0.8011 - loss: 0.4801 - val_accuracy: 0.7991 - val_loss: 0.4818
Epoch 6/50
25357/25357 ━━━━━━━━━━━━━━━━━━━━ 67s 3ms/step - accuracy: 0.7999 - loss: 0.4816 - val_accuracy: 0.7983 - val_loss: 0.4820
Epoch 7/50
25357/25357 ━━━━━━━━━━━━━━━━━━━━ 68s 3ms/step - accuracy: 0.8004 - loss: 0.4808 - val_accuracy: 0.7998 - val_loss: 0.4815
Epoch 8/50
25357/25357 ━━━━━━━━━━━━━━━━━━━━ 70s 3ms/step - accuracy: 

In [81]:
loss, acc = model.evaluate(x_test, y_test)
print("Loss:", loss, "Accuracy:", acc)

12679/12679 ━━━━━━━━━━━━━━━━━━━━ 24s 2ms/step - accuracy: 0.8046 - loss: 0.4735
Loss: 0.4734399914741516 Accuracy: 0.8046892285346985


In [92]:
#Perfil 1: Hombre ladino de Santa Rosa
Perfil1 = np.array([[6,4,1]])
res = model.predict(Perfil1)
print(res)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
[[0.24153416]]


In [93]:
#Perfil 2: Mujer maya de Sacatepéquez
Perfil2 = np.array([[3,1,2]])
res = model.predict(Perfil2)
print(res)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
[[0.21957833]]


In [94]:
#Perfil 3: Mujer ladina de Petén
Perfil3 = np.array([[17,4,2]])
res = model.predict(Perfil3)
print(res)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
[[0.18991813]]
